# Using Plotly and TA-Lib to make financial charts

## Import required libraries

TA-Lib is an industry-standard package used for technical analysis, and has Python bindings.

Note that Python TA-Lib wrappers reqires both pip install and the underlying library:

- Use brew install ta-lib on Mac
- Download installers for Windows and Linux

In [29]:
import copy
import numpy as np
import pandas as pd
import cufflinks as cf # Optional

import plotly.plotly as py
from plotly.graph_objs import *

import pandas_datareader.data as web
import talib

## Fetch end of day prices from Yahoo

In [9]:
ticker = 'AAPL' 

df = web.get_data_yahoo(ticker) # Feed AAPL from Yahoo stock data
df.shape # 1789 most recent trading days

price = df['Adj Close'] # Get Yahoo price as DataFrame
ndprice = price.values # Convert to NumPy ndarray (TA-Lib has no DataFrame or Series support)

## Test our first TA-Lib indicator

Most indicators are of format 

#### OUTPUT, [OUTPUT2] = talib.INDICATOR(close, [timeperiod=TIMEPERIOD, **args])

Due to this syntax consistency, TA-Lib's indicators are extremely easy to use as opposed to Pandas' sometimes inconsitent library, like .rolling() vs .rolling_mean() for instance. TA-Lib's calculations are additionally not done in Pyhton, which results in additional speed incease.

Read full reference here: http://mrjbq7.github.io/ta-lib/

In [10]:
MA30 = talib.MA(ndprice) # Default window of 30D

## Quick plot

In [11]:
output = {
    'Adj Close': price,
    'MA30': MA30,
}

output = pd.DataFrame(output) # Remake a new DataFrame
output.iplot() # Quick Plotly chart with Cufflinks (using .iplot() method on a df)

## More indicators, Golden Cross

Let's make the golden cross (the 50SMA and 200SMA crossover)
The Golden Cross is one of the best known technical indicators, and is often interpreted as a sign of long-term price movement.

In [12]:
MA50 = talib.MA(ndprice, timeperiod=50)
MA200 = talib.MA(ndprice, timeperiod=200)

output = {
    'Adj Price': price,
    'MA50': MA50,
    'MA200': MA200
}

output = pd.DataFrame(output)
output.iplot()

## Making Candlestick Charts

For customizability and advanced features, one must use Plotly directly as Cufflinks isn't well suited for full-on chart customization.

In [13]:
volume = df['Volume'] 

# Sanitize for split, dividend, etc. as O, # L, H, C not adjuted on Yahoo
ratio = df['Close']/df['Adj Close'] 
#del df['Adj Close']
df = df.div(ratio, axis='index') # Elementwise Pandas divide

trace1 = Candlestick(
    x = df.index,
    open = df['Open'] ,
    low = df['Low'] ,
    high = df['High'] ,
    close = df['Close'],
    name = ticker,
)

trace2 = Scatter(
    x = df.index,
    y = MA50, # Plotly accepts either a DataFrame or a NumPy array 
    name = 'MA50', 
    mode = 'lines',
)

trace3 = copy.copy(trace2) # Copy trace2
trace3.y = MA200
trace3.name = 'MA200'

data = [trace1, trace2, trace3]
py.iplot(data)

## Adding volume subplot

Though leaving volume unadjusted will distort the numbers somewhat, it is customary to not adjust volume as adjusted volume tends to spike even more after a split.

In [14]:
# Data
trace1 = Candlestick(
    x = df.index,
    open = df['Open'] ,
    low = df['Low'] ,
    high = df['High'] ,
    close = df['Close'],
    name = ticker,
    yaxis = 'y1', # Main plot
)

trace2 = Scatter(
    x = df.index,
    y = MA50,
    name = 'MA50',
    mode = 'lines',
    yaxis = 'y1', # Main plot
)

trace3 = copy.copy(trace2) # Copy trace2
trace3.y = MA200
trace3.name = 'MA200'

trace4 = Bar(
    x = df.index,
    y = volume, # Non-adjusted volume, if you adjust you see spikes
    name = 'Volume',
    yaxis = 'y2', # Subplot
)

# Layout
layout = Layout(
    yaxis = dict(
        domain = [0.25, 1],
    ),
    yaxis2 = dict(
        domain = [0, 0.2],
    ),
    xaxis = dict(
        rangeslider = dict(
            visible = False,
        ),
    ),
)

data = [trace1, trace2, trace3, trace4]
figure = Figure(data = data, layout = layout)
py.iplot(figure)

## Styling everything up

In [31]:
# Data
trace1 = Candlestick(
    x = df.index,
    open = df['Open'] ,
    low = df['Low'] ,
    high = df['High'] ,
    close = df['Close'],
    name = ticker,
    yaxis = 'y1', # Main plot
)

trace2 = Scatter(
    x = df.index,
    y = MA50,
    name = 'MA50',
    mode = 'lines',
    line = dict(color = '#1480DC'),
    yaxis = 'y1', # Main plot
)

trace3 = copy.copy(trace2) # Copy trace2
trace3.y = MA200
trace3.name = 'MA200'

# Either green volume bar if close > open or red bar if the opposite
colors = ["green" if (value - df['Open'].values[i]) >= 0 else "red"
          for i, value in enumerate(df['Close'].values)]

trace4 = Bar(
    x = df.index,
    y = volume, # Non-adjusted volume, if you adjust you see spikes
    name = 'Volume',
    marker = dict(color = colors),
    yaxis = 'y2', # Subplot,
)

# Layout
layout = Layout(
    
    # GENERAL
    title = ticker,
    margin = dict(
        t = 50,
        l = 50,
        b = 50,
        r = 50,
        pad = 0, 
    ),
    font = dict(family = 'overpass'),
    showlegend = False,
    plot_bgcolor = '#FAFAFA',
    paper_bgcolor = '#F5F6F9',
    
    # AXIS
    yaxis = dict(
        domain = [0.25, 1],
        type = 'log',
    ),
    yaxis2 = dict(
        domain = [0, 0.2],
    ),
    xaxis = dict(
        rangeslider = dict(
            visible = False,
        ),
    ),
    
)

data = [trace1, trace2, trace3, trace4]
figure = Figure(data = data, layout = layout)
py.iplot(figure)